In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Load Profiles

Notes:

l1_p , l2_p , l3_p : mean power in W. To obtain total energy in Wh consumed during the interval (depends on the granularity):

 - Energy = (l1_p + l2_p + l3_p) / granularity 

Depending on the usage and displayed data, the power of the 3 phases ( l1_p , l2_p , l3_p ) need to be added to form a single value.

In [ ]:
day1 = '5fe3412c923d596335e69d4c-20210808-v0.0.1_SE05000139_Kaeltemaschine.csv' ## Sunday
day2 = '5fe3412c923d596335e69d4c-20210809-v0.0.1_SE05000139_Kaeltemaschine.csv' ## Monday
day3 = '5fe3412c923d596335e69d4c-20210810-v0.0.1_SE05000139_Kaeltemaschine.csv' ## Tuesday
day1 = pd.read_csv('/Users/wastechs/Documents/data/Kaeltemaschine/{}'.format(day1))
day2 = pd.read_csv('/Users/wastechs/Documents/data/Kaeltemaschine/{}'.format(day2))
day3 = pd.read_csv('/Users/wastechs/Documents/data/Kaeltemaschine/{}'.format(day3))

In [ ]:
day1.head()

In [ ]:
def epoch_convert(df):
    df.t = pd.to_datetime(df.t, unit='s')
    df.set_index(df.t, inplace=True)
    del df['t']
    return df

In [ ]:
day1 = epoch_convert(day1)
day2 = epoch_convert(day2)
day3 = epoch_convert(day3)

In [ ]:
day1.shape, day2.shape, day3.shape

In [ ]:
day1.index.min(), day1.index.max(), day2.index.min(), day2.index.max(), day3.index.min(), day3.index.max()

In [ ]:
day1.head(6)

In [ ]:
day1.index[3] - day1.index[0]

In [ ]:
def kwatt(df):
    average = pd.DataFrame(df[['P', 'I', 'V', 'PF']].groupby(df.index).mean())
    average['kW'] = (average['V'] * average['I'] * average['PF'] * np.sqrt(3)) / 1000

    return average

In [ ]:
day1_kw = kwatt(day1)
day2_kw = kwatt(day2)
day3_kw = kwatt(day3)

In [ ]:
def decompose_phases(df):
    df_1 = df[df.L == 1.0]
    df_2 = df[df.L == 2.0]
    df_3 = df[df.L == 3.0]

    return df_1, df_2, df_3

In [ ]:
day1_phase1, day1_phase2, day1_phase3 = decompose_phases(day1)
day2_phase1, day2_phase2, day2_phase3 = decompose_phases(day2)
day3_phase1, day3_phase2, day3_phase3 = decompose_phases(day3)

In [ ]:
def load_plotter(df, feat, resample=None):
    
    if resample != None:
        plt.figure(figsize=(16, 6))
        plt.plot(df.resample(resample).sum().index, df[feat].resample(resample).mean())
        plt.title('Kaeltemaschine')
        plt.xlabel('Time = M-d-H')
        plt.ylabel('Power')
        plt.show()

    else:
        plt.figure(figsize=(16, 6))
        plt.plot(df.index, df[feat])
        plt.title('Kaeltemaschine')
        plt.xlabel('Time = M-d-H')
        plt.ylabel('{}'.format(feat))
        plt.show()

### Original Resolution

Tuesday:
- Idle / standby mode seems to be occuring from 21:00 until about 7:00
- Normal operations from 7:00 until 19:00 and standby mode again until 7:00
- Three peaks occuring around mid-day and in the afteroon are worth investigating
    - It would be interesting to know what Gassmann was doing in/with the fridge during these time periods?
    - Moving products inside, taking products out. . .

In [ ]:
load_plotter(day3_phase1, 'P')

Monday:
- Idle / standby mode seems to be occuring from 21:00 until about 8:00
- Normal operations from 8:00 until 19:00 and standby mode again until 7:00
- The difference with this load profile is there are no peaks occuring. Rather a steady state "normal" operation mode throughout the business hours with a slight linear trend until noon and then a slight linear decreasing trend until ~17:00

In [ ]:
load_plotter(day2_phase1, 'P')

Sunday:
 - Idle / standby mode seems to be occuring for the majority of the day with cycling periods
 - Power consumption increases slightly throughout the day until about ~13:00 and then decreases
    - Possibly due to increasing ambient temperatures?

In [ ]:
load_plotter(day1_phase1, 'P')

Investigating Monday idle mode:
 - How long do these peaks last?

In [ ]:
# Idle mode for Monday
load_plotter(day2_phase1['2021-08-08 21:59': '2021-08-09 8:30:00'], 'P')

In [ ]:
# 
load_plotter(day2_phase1['2021-08-09 14:00:00': '2021-08-09 21:30:00'], 'P')

In [ ]:
# What's signal and what's noise here?
load_plotter(day2_phase1['2021-08-09 17:27:30': '2021-08-09 17:28:15'], 'P')

Investigating the first peak on Tuesday:

- What's signal and what's noise here?
    - After the initial jump, consumption falls down to a consistent level for ~5 minutes and then falls down to "normal" operating hour levels again

In [ ]:
load_plotter(day3_phase1['2021-08-10 11:25': '2021-08-10 11:30'], 'P') ## instant. power

## spike mainly comes from the current becasue it draws more current

In [ ]:
day1.describe().T

## 15 minute Resolution

In [ ]:
load_plotter(day3_phase1, 'P', '1H')

In [ ]:
load_plotter(day2_phase1, 'P', '1H')

In [ ]:
load_plotter(day1_phase1, 'P', '1H')

In [ ]:
sns.kdeplot(day1_phase1.P)
sns.kdeplot(day1_phase2.P)
sns.kdeplot(day1_phase3.P)